In [56]:
import re
import csv
import time
import requests
import datetime
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [59]:
column_array=[["No","Group","Date","Winning","Match1","Match1","Match1","Match2","Match2","Match2","Match3","Match3","Match3","Match4","Match4","Match4","Match5","Match5","Match5"],["","","","","Home","Away","W/L","Home","Away","W/L","Home","Away","W/L","Home","Away","W/L","Home","Away","W/L"]]
column_tuples = list(zip(*column_array))
column = pd.MultiIndex.from_tuples(column_tuples)
df=pd.DataFrame(columns=column)
club_id = pd.read_csv("./club_and_id.csv")

for n in range(1200,1210):
    url =f'https://toto.rakuten.co.jp/toto/result/{n:04}/'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    try:
        for group in ["A","B"]:
            date = soup.select(f"#minitotoBlock{group} .tbl-basic-day td")[0].string
            wining = soup.select(f"#minitotoBlock{group} .tbl-basic-money tbody td")[0].string
            toto_data = [n, group, date,wining]

            match = soup.select(f"#minitotoBlock{group} .tbl-result tbody tr")
            for i in range(5):
                home_ja = match[i].select("td")[1].string
                away_ja = match[i].select("td")[3].string

                home = ""
                away = ""

                for index,row in club_id.iterrows():
                    if home_ja == row["club_ja"]:
                        home = row["club"]
                    elif away_ja == row["club_ja"]:
                        away = row["club"]

                match_data =[home, away, int(match[i].select("td")[5].string)]
                toto_data.extend(match_data)

            if not "" in toto_data:
                df.loc[f"{n}_{group}"]= toto_data
    
    except IndexError:
        pass
    except ValueError:
        pass
    
    time.sleep(1)

In [60]:
df

No Group            Date  Winning                Match1  \
                                                             Home   
1200_A  1200     A   2020年11月4日(水)  19,921円        vegalta-sendai   
1204_A  1204     A  2020年11月17日(火)  75,846円   sanfrecce-hiroshima   
1206_A  1206     A  2020年11月25日(水)  12,793円     consadole-sapporo   
1207_A  1207     A  2020年11月27日(金)  17,627円     kawasaki-frontale   
1208_A  1208     A   2020年12月1日(火)  12,754円  nagoya-grampus-eight   

                                              Match2                          \
                       Away W/L                 Home                Away W/L   
1200_A  sanfrecce-hiroshima   0         oita-trinita  urawa-red-diamonds   0   
1204_A          yokohama-fc   0      shimizu-s-pulse        cerezo-osaka   1   
1206_A      shimizu-s-pulse   1         oita-trinita   kawasaki-frontale   1   
1207_A          gamba-osaka   1      kashima-antlers      kashiwa-reysol   2   
1208_A         oita-trinita   0  sanfrecce-hiroshima   consadole-sapporo   0   

                 Match3                                    Match4  \
                   Home             Away W/L                 Home   
1200_A  shonan-bellmare      yokohama-fc   1      kashima-antlers   
1204_A      gamba-osaka   vegalta-sendai   2  yokohama-fa-marinos   
1206_A   vegalta-sendai  kashima-antlers   2         cerezo-osaka   
1207_A     oita-trinita     cerezo-osaka   2      shonan-bellmare   
1208_A  shimizu-s-pulse  shonan-bellmare   0         cerezo-osaka   

                                                Match5                          
                        Away W/L                  Home                Away W/L  
1200_A  nagoya-grampus-eight   2           gamba-osaka   consadole-sapporo   1  
1204_A    urawa-red-diamonds   1       kashima-antlers   kawasaki-frontale   0  
1206_A   sanfrecce-hiroshima   2  nagoya-grampus-eight     shonan-bellmare   1  
1207_A   sanfrecce-hiroshima   0            sagan-tosu      vegalta-sendai   2  
1208_A           yokohama-fc   1       kashima-antlers  urawa-red-diamonds   1

In [61]:
df["Date"] = df["Date"].replace('\(.+\)','',regex=True)
df["Date"] = pd.to_datetime(df["Date"], format="%Y年%m月%d日")

In [62]:
df

No Group       Date  Winning                Match1  \
                                                        Home   
1200_A  1200     A 2020-11-04  19,921円        vegalta-sendai   
1204_A  1204     A 2020-11-17  75,846円   sanfrecce-hiroshima   
1206_A  1206     A 2020-11-25  12,793円     consadole-sapporo   
1207_A  1207     A 2020-11-27  17,627円     kawasaki-frontale   
1208_A  1208     A 2020-12-01  12,754円  nagoya-grampus-eight   

                                              Match2                          \
                       Away W/L                 Home                Away W/L   
1200_A  sanfrecce-hiroshima   0         oita-trinita  urawa-red-diamonds   0   
1204_A          yokohama-fc   0      shimizu-s-pulse        cerezo-osaka   1   
1206_A      shimizu-s-pulse   1         oita-trinita   kawasaki-frontale   1   
1207_A          gamba-osaka   1      kashima-antlers      kashiwa-reysol   2   
1208_A         oita-trinita   0  sanfrecce-hiroshima   consadole-sapporo   0   

                 Match3                                    Match4  \
                   Home             Away W/L                 Home   
1200_A  shonan-bellmare      yokohama-fc   1      kashima-antlers   
1204_A      gamba-osaka   vegalta-sendai   2  yokohama-fa-marinos   
1206_A   vegalta-sendai  kashima-antlers   2         cerezo-osaka   
1207_A     oita-trinita     cerezo-osaka   2      shonan-bellmare   
1208_A  shimizu-s-pulse  shonan-bellmare   0         cerezo-osaka   

                                                Match5                          
                        Away W/L                  Home                Away W/L  
1200_A  nagoya-grampus-eight   2           gamba-osaka   consadole-sapporo   1  
1204_A    urawa-red-diamonds   1       kashima-antlers   kawasaki-frontale   0  
1206_A   sanfrecce-hiroshima   2  nagoya-grampus-eight     shonan-bellmare   1  
1207_A   sanfrecce-hiroshima   0            sagan-tosu      vegalta-sendai   2  
1208_A           yokohama-fc   1       kashima-antlers  urawa-red-diamonds   1

In [63]:
df=df.set_index(["No","Group"], drop=True)

In [64]:
df

Date  Winning                Match1                           \
                                                Home                 Away W/L   
No   Group                                                                      
1200 A     2020-11-04  19,921円        vegalta-sendai  sanfrecce-hiroshima   0   
1204 A     2020-11-17  75,846円   sanfrecce-hiroshima          yokohama-fc   0   
1206 A     2020-11-25  12,793円     consadole-sapporo      shimizu-s-pulse   1   
1207 A     2020-11-27  17,627円     kawasaki-frontale          gamba-osaka   1   
1208 A     2020-12-01  12,754円  nagoya-grampus-eight         oita-trinita   0   

                         Match2                                   Match3  \
                           Home                Away W/L             Home   
No   Group                                                                 
1200 A             oita-trinita  urawa-red-diamonds   0  shonan-bellmare   
1204 A          shimizu-s-pulse        cerezo-osaka   1      gamba-osaka   
1206 A             oita-trinita   kawasaki-frontale   1   vegalta-sendai   
1207 A          kashima-antlers      kashiwa-reysol   2     oita-trinita   
1208 A      sanfrecce-hiroshima   consadole-sapporo   0  shimizu-s-pulse   

                                              Match4                        \
                       Away W/L                 Home                  Away   
No   Group                                                                   
1200 A          yokohama-fc   1      kashima-antlers  nagoya-grampus-eight   
1204 A       vegalta-sendai   2  yokohama-fa-marinos    urawa-red-diamonds   
1206 A      kashima-antlers   2         cerezo-osaka   sanfrecce-hiroshima   
1207 A         cerezo-osaka   2      shonan-bellmare   sanfrecce-hiroshima   
1208 A      shonan-bellmare   0         cerezo-osaka           yokohama-fc   

                              Match5                          
           W/L                  Home                Away W/L  
No   Group                                                    
1200 A       2           gamba-osaka   consadole-sapporo   1  
1204 A       1       kashima-antlers   kawasaki-frontale   0  
1206 A       2  nagoya-grampus-eight     shonan-bellmare   1  
1207 A       0            sagan-tosu      vegalta-sendai   2  
1208 A       1       kashima-antlers  urawa-red-diamonds   1

In [65]:
df["Winning"]=df["Winning"].str.replace(",","")
df["Winning"]=df["Winning"].str.replace("円","").astype(int)

In [66]:
df

Date Winning                Match1                           \
                                               Home                 Away W/L   
No   Group                                                                     
1200 A     2020-11-04   19921        vegalta-sendai  sanfrecce-hiroshima   0   
1204 A     2020-11-17   75846   sanfrecce-hiroshima          yokohama-fc   0   
1206 A     2020-11-25   12793     consadole-sapporo      shimizu-s-pulse   1   
1207 A     2020-11-27   17627     kawasaki-frontale          gamba-osaka   1   
1208 A     2020-12-01   12754  nagoya-grampus-eight         oita-trinita   0   

                         Match2                                   Match3  \
                           Home                Away W/L             Home   
No   Group                                                                 
1200 A             oita-trinita  urawa-red-diamonds   0  shonan-bellmare   
1204 A          shimizu-s-pulse        cerezo-osaka   1      gamba-osaka   
1206 A             oita-trinita   kawasaki-frontale   1   vegalta-sendai   
1207 A          kashima-antlers      kashiwa-reysol   2     oita-trinita   
1208 A      sanfrecce-hiroshima   consadole-sapporo   0  shimizu-s-pulse   

                                              Match4                        \
                       Away W/L                 Home                  Away   
No   Group                                                                   
1200 A          yokohama-fc   1      kashima-antlers  nagoya-grampus-eight   
1204 A       vegalta-sendai   2  yokohama-fa-marinos    urawa-red-diamonds   
1206 A      kashima-antlers   2         cerezo-osaka   sanfrecce-hiroshima   
1207 A         cerezo-osaka   2      shonan-bellmare   sanfrecce-hiroshima   
1208 A      shonan-bellmare   0         cerezo-osaka           yokohama-fc   

                              Match5                          
           W/L                  Home                Away W/L  
No   Group                                                    
1200 A       2           gamba-osaka   consadole-sapporo   1  
1204 A       1       kashima-antlers   kawasaki-frontale   0  
1206 A       2  nagoya-grampus-eight     shonan-bellmare   1  
1207 A       0            sagan-tosu      vegalta-sendai   2  
1208 A       1       kashima-antlers  urawa-red-diamonds   1

In [67]:
df[("Match1","W/L")]

No    Group
1200  A        0
1204  A        0
1206  A        1
1207  A        1
1208  A        0
Name: (Match1, W/L), dtype: object

In [68]:
# for i in range(1,6):
#     try:
#         df[f"Match{i}","W/L"]=df[f"Match{i}","W/L"].astype(int)
#     except ValueError:
#         pass

In [69]:
df[("Match3","W/L")]

No    Group
1200  A        1
1204  A        2
1206  A        2
1207  A        2
1208  A        0
Name: (Match3, W/L), dtype: object

In [70]:
df.insert(2, ( 'Match1', 'MatchID'),np.nan)
df.insert(6, ( 'Match2', 'MatchID'),np.nan)
df.insert(10, ( 'Match3', 'MatchID'),np.nan)
df.insert(14, ( 'Match4', 'MatchID'),np.nan)
df.insert(18, ( 'Match5', 'MatchID'),np.nan)

In [71]:
df[("Match1","MatchID")]

No    Group
1200  A       NaN
1204  A       NaN
1206  A       NaN
1207  A       NaN
1208  A       NaN
Name: (Match1, MatchID), dtype: float64

In [72]:
df_match_data = pd.read_csv("./match_data_yearly/all_years.csv")
df_match_data["Date"] = pd.to_datetime(df_match_data["Date"])

In [73]:
df_match_data =df_match_data.set_index(["Date"], drop=True)
df_match_data

,ID,Year,Sec,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,Stadium,Attendances,HomeElo,AwayElo,HomeED,AwayED
Date,,,,,,,,,,,,,,,,
2006-03-04,6010903,2006,1,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,万博,20916,1500,1500,0,0
2006-03-05,6012807,2006,1,ventforet-kofu,shimizu-s-pulse,28,7,0,2,2,小瀬,14277,1500,1500,0,0
2006-03-05,6012231,2006,1,fc-tokyo,oita-trinita,22,31,2,0,1,味スタ,22531,1500,1500,0,0
2006-03-05,6011323,2006,1,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,静岡,28564,1500,1500,0,0
2006-03-05,6010820,2006,1,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,瑞穂陸,17199,1500,1500,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-19,20340120,2020,34,kashima-antlers,cerezo-osaka,1,20,1,1,0,カシマ,11251,1594,1560,4,-6
2020-12-19,20340810,2020,34,nagoya-grampus-eight,sanfrecce-hiroshima,8,10,1,0,1,豊田ス,16825,1515,1524,9,-13
2020-12-19,20343331,2020,34,sagan-tosu,oita-trinita,33,31,2,2,0,駅スタ,8966,1453,1458,17,7


In [74]:
df_match_data_20 = df_match_data["2020"]

In [75]:
df_match_data_20[(df_match_data_20["Home"]=="oita-trinita") & (df_match_data_20["Away"] == "vissel-kobe")]

,ID,Year,Sec,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,Stadium,Attendances,HomeElo,AwayElo,HomeED,AwayED
Date,,,,,,,,,,,,,,,,
2020-07-11,20043118,2020,4,oita-trinita,vissel-kobe,31,18,1,1,0,昭和電ド,4263,1441,1497,20,0


In [76]:
np.array(df_match_data_20[(df_match_data_20["Home"]=="oita-trinita") & (df_match_data_20["Away"] == "vissel-kobe")]["ID"].values[0])

array(20043118)

In [77]:
for index,row in df.iterrows():
    for i in range(1,6):
        home = row[f"Match{i}"]["Home"]
        away = row[f"Match{i}"]["Away"]
        df[f"Match{i}","W/L"]=df[f"Match{i}","W/L"].astype(int)
        try:
            match_id = df_match_data_20[(df_match_data_20["Home"]==home) & (df_match_data_20["Away"] == away)]["ID"].values[0]
            df.at[index, (f"Match{i}","MatchID")] = match_id
        except IndexError:
            pass        

In [78]:
df[("Match3","W/L")]

No    Group
1200  A        1
1204  A        2
1206  A        2
1207  A        2
1208  A        0
Name: (Match3, W/L), dtype: int64

In [79]:
df

Date Winning      Match1                        \
                                  MatchID                  Home   
No   Group                                                        
1200 A     2020-11-04   19921  20255410.0        vegalta-sendai   
1204 A     2020-11-17   75846  20271034.0   sanfrecce-hiroshima   
1206 A     2020-11-25   12793  20281407.0     consadole-sapporo   
1207 A     2020-11-27   17627  20292109.0     kawasaki-frontale   
1208 A     2020-12-01   12754  20300831.0  nagoya-grampus-eight   

                                         Match2                       \
                           Away W/L     MatchID                 Home   
No   Group                                                             
1200 A      sanfrecce-hiroshima   0  20253103.0         oita-trinita   
1204 A              yokohama-fc   0  20270720.0      shimizu-s-pulse   
1206 A          shimizu-s-pulse   1  20283121.0         oita-trinita   
1207 A              gamba-osaka   1  20290111.0      kashima-antlers   
1208 A             oita-trinita   0  20301014.0  sanfrecce-hiroshima   

                                    ...           Match3          Match4  \
                          Away W/L  ...             Away W/L     MatchID   
No   Group                          ...                                    
1200 A      urawa-red-diamonds   0  ...      yokohama-fc   1  20250108.0   
1204 A            cerezo-osaka   1  ...   vegalta-sendai   2  20270503.0   
1206 A       kawasaki-frontale   1  ...  kashima-antlers   2  20282010.0   
1207 A          kashiwa-reysol   2  ...     cerezo-osaka   2  20291210.0   
1208 A       consadole-sapporo   0  ...  shonan-bellmare   0  20302034.0   

                                                               Match5  \
                           Home                  Away W/L     MatchID   
No   Group                                                              
1200 A          kashima-antlers  nagoya-grampus-eight   2  20250914.0   
1204 A      yokohama-fa-marinos    urawa-red-diamonds   1  20270121.0   
1206 A             cerezo-osaka   sanfrecce-hiroshima   2  20280812.0   
1207 A          shonan-bellmare   sanfrecce-hiroshima   0  20293354.0   
1208 A             cerezo-osaka           yokohama-fc   1  20300103.0   

                                                          
                            Home                Away W/L  
No   Group                                                
1200 A               gamba-osaka   consadole-sapporo   1  
1204 A           kashima-antlers   kawasaki-frontale   0  
1206 A      nagoya-grampus-eight     shonan-bellmare   1  
1207 A                sagan-tosu      vegalta-sendai   2  
1208 A           kashima-antlers  urawa-red-diamonds   1  

[5 rows x 22 columns]

In [80]:
df=df.dropna()

In [81]:
# pd.options.mode.chained_assignment = None # これを有効にすると､SettingWithCopyWarningをオフできる(非推奨)
for i in range(1,6):
    df[f"Match{i}","MatchID"] = df[f"Match{i}","MatchID"].astype(int)

In [82]:
df[("Match1","MatchID")]

No    Group
1200  A        20255410
1204  A        20271034
1206  A        20281407
1207  A        20292109
1208  A        20300831
Name: (Match1, MatchID), dtype: int64

In [83]:
df

Date Winning    Match1                        \
                                MatchID                  Home   
No   Group                                                      
1200 A     2020-11-04   19921  20255410        vegalta-sendai   
1204 A     2020-11-17   75846  20271034   sanfrecce-hiroshima   
1206 A     2020-11-25   12793  20281407     consadole-sapporo   
1207 A     2020-11-27   17627  20292109     kawasaki-frontale   
1208 A     2020-12-01   12754  20300831  nagoya-grampus-eight   

                                       Match2                       \
                           Away W/L   MatchID                 Home   
No   Group                                                           
1200 A      sanfrecce-hiroshima   0  20253103         oita-trinita   
1204 A              yokohama-fc   0  20270720      shimizu-s-pulse   
1206 A          shimizu-s-pulse   1  20283121         oita-trinita   
1207 A              gamba-osaka   1  20290111      kashima-antlers   
1208 A             oita-trinita   0  20301014  sanfrecce-hiroshima   

                                    ...           Match3        Match4  \
                          Away W/L  ...             Away W/L   MatchID   
No   Group                          ...                                  
1200 A      urawa-red-diamonds   0  ...      yokohama-fc   1  20250108   
1204 A            cerezo-osaka   1  ...   vegalta-sendai   2  20270503   
1206 A       kawasaki-frontale   1  ...  kashima-antlers   2  20282010   
1207 A          kashiwa-reysol   2  ...     cerezo-osaka   2  20291210   
1208 A       consadole-sapporo   0  ...  shonan-bellmare   0  20302034   

                                                             Match5  \
                           Home                  Away W/L   MatchID   
No   Group                                                            
1200 A          kashima-antlers  nagoya-grampus-eight   2  20250914   
1204 A      yokohama-fa-marinos    urawa-red-diamonds   1  20270121   
1206 A             cerezo-osaka   sanfrecce-hiroshima   2  20280812   
1207 A          shonan-bellmare   sanfrecce-hiroshima   0  20293354   
1208 A             cerezo-osaka           yokohama-fc   1  20300103   

                                                          
                            Home                Away W/L  
No   Group                                                
1200 A               gamba-osaka   consadole-sapporo   1  
1204 A           kashima-antlers   kawasaki-frontale   0  
1206 A      nagoya-grampus-eight     shonan-bellmare   1  
1207 A                sagan-tosu      vegalta-sendai   2  
1208 A           kashima-antlers  urawa-red-diamonds   1  

[5 rows x 22 columns]

In [84]:
# df.to_csv("./toto_data/2020.csv")